In [1]:
import pandas as pd
import datetime
import time
import re

---
Functions

---

In [2]:
def fix_date_format(dataframe, date_columns):
    def convert_date_format(date_str):
        if isinstance(date_str, str):
            try:
                date_obj = datetime.datetime.strptime(date_str, '%m-%d-%Y')
            except ValueError:
                try:
                    date_obj = datetime.datetime.strptime(date_str, '%m/%d/%Y')
                except ValueError:
                    return None
            return date_obj.strftime('%m-%d-%Y') if date_obj else None
        else:
            return None

    for date_column in date_columns:
        dataframe[date_column] = dataframe[date_column].apply(lambda x: convert_date_format(x))


In [3]:
def clean_city_names(dataframe):
    def process_city(city):
        if isinstance(city, str):
            city = city.rstrip(',')
            city = re.sub(r'^B.*CANCOUR$', 'BECANCOUR', city)
            city = re.sub(r'^Rivi.*-Rouge$', 'RIVIERE-ROUGE', city)
        return city

    dataframe['CITY_DEALER'] = dataframe['CITY_DEALER'].apply(process_city)
    return dataframe

In [4]:
def clean_dealer_dataset(df_dealer):
    # Remove duplicates
    df_dealer.drop_duplicates(keep='first', inplace=True)

    # Filter for 'US' and 'CA' in 'COUNTRY_CODE'
    df_dealer = df_dealer[(df_dealer['COUNTRY_CODE'] == 'US') | (df_dealer['COUNTRY_CODE'] == 'CA')]

    # Fill missing values in 'CITY_DEALER' with 'no city found'
    df_dealer['CITY_DEALER'].fillna('no city found', inplace=True)

    # Select columns of interest
    df_dealer = df_dealer[['CUSTOMER_NUMBER', 'CITY_DEALER', 'COUNTRY_CODE', 'STATE_CODE']]

    # Convert 'CUSTOMER_NUMBER' to integer
    df_dealer['CUSTOMER_NUMBER'] = df_dealer['CUSTOMER_NUMBER'].astype(int)

    # Define a region mapping dictionary
    region_mapping = {
        1: ["AB", "BC", "MB", "NB", "NF", "NS", "NT", "NU", "ON", "PE", "QC", "SK", "YT"],
        2: ["AL", "CT", "DE", "DC", "FL", "GA", "MA", "MD", "ME", "NC", "NH", "NJ", "NY", "PA", "PR", "RI", "SC", "VA", "VT", "WV"],
        3: ["AR", "IA", "IL", "IN", "KS", "KY", "LA", "MI", "MN", "MO", "MS", "ND", "NE", "OH", "SD", "TN", "WI"],
        4: ["AK", "AZ", "CA", "CO", "HI", "ID", "MT", "NM", "NV", "OK", "OR", "TX", "UT", "WA", "WY"]
    }

    # Map 'REGION_CODE' based on 'STATE_CODE'
    df_dealer['REGION_CODE'] = df_dealer['STATE_CODE'].apply(
        lambda state: next((region for region, states in region_mapping.items() if state in states), 5)
    )

    return df_dealer

In [5]:
def clean_retail_dataset(df_retail):
    # Remove 'x' characters from 'REG_DEALER_NUMBER'
    df_retail['REG_DEALER_NUMBER'] = df_retail['REG_DEALER_NUMBER'].str.replace('x', '', regex=True)

    # Set 'RETAIL' column to 1
    df_retail['RETAIL'] = 1

    # Convert 'REG_DEALER_NUMBER' to integer
    df_retail['REG_DEALER_NUMBER'] = df_retail['REG_DEALER_NUMBER'].astype(int)

    return df_retail

In [6]:
def merge_datasets(df_dealer, df_retail):
    # Merge the DataFrames using an inner join
    df_merged = pd.merge(df_retail, df_dealer, left_on='REG_DEALER_NUMBER', right_on='CUSTOMER_NUMBER', how='inner')

    return df_merged

---
End of function definitions

---

---
Setting parameters

---

In [7]:
dealer_data_path = "/content/sample_data/DEALER.csv"
retail_data_path = "/content/sample_data/RETAIL_SALES.csv"

In [8]:
df_dealer = pd.read_csv(dealer_data_path)
df_retail = pd.read_csv(retail_data_path)

<ipython-input-8-8a4cc50ed4c7>:2: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df_retail = pd.read_csv(retail_data_path)


In [9]:
dealer_date_columns = ['ATV_CREATION_DATE','PWC_CREATION_DATE',
                       'SNOW_CREATION_DATE','THREE_W_CREATION_DATE',
                       'SSV_CREATION_DATE']

retail_date_columns = ['DELIVERY_DATE','FLOORING_END_DATE',
                       'LAST_STORAGE_DATE','PAID_OFF_DATE',
                       'REGISTRATION_DATE','DATE_OF_SALE',
                       'INVOICE_DATE']

---
End of setting parameters

---

---
Final Script

---

In [10]:
def final_script(df_dealer, df_retail, dealer_date_columns, retail_date_columns):

    print("Number of rows and columns in the dealer dataset")
    print("# of rows: {}".format(df_dealer.shape[0]))
    print("# of columns: {}\n".format(df_dealer.shape[1]))

    print("----//----//----//----//----//----//----//----")
    print("Number of rows and columns in the retail dataset")
    print("# of rows: {}".format(df_retail.shape[0]))
    print("# of columns: {}".format(df_retail.shape[1]))

    # Fix date format
    start_time = time.time()
    print("\nFixing the date for each dataset")
    fix_date_format(df_dealer, dealer_date_columns)
    fix_date_format(df_retail, retail_date_columns)
    end_time = time.time()
    print(f"Finished the date fixing functions in {round(end_time-start_time,2)} seconds \n")
    print("----//----//----//----//----//----//----//----")

    start_time = time.time()
    print("Adjusting city names in the dealer dataset")
    # Clean and fix city names
    df_dealer = clean_city_names(df_dealer)
    end_time = time.time()
    print(f"Finished the city adjusting functions in  {round(end_time-start_time,2)} seconds \n")
    print("----//----//----//----//----//----//----//----")

    start_time = time.time()
    print("General cleaning in both datasets")
    # Clean the dealer and retail datasets
    df_dealer = clean_dealer_dataset(df_dealer)
    df_retail = clean_retail_dataset(df_retail)
    end_time = time.time()
    print(f"Finished the general cleaning in  {round(end_time-start_time,2)} seconds \n")
    print("----//----//----//----//----//----//----//----")

    # Merge datasets
    start_time = time.time()
    print("Merging datasets")
    df_merged = merge_datasets(df_dealer, df_retail)
    end_time = time.time()
    print(f"Finished the data merging in  {round(end_time-start_time,2)} seconds \n")

    return df_merged

In [11]:
df_merged = final_script(df_dealer, df_retail, dealer_date_columns, retail_date_columns)

Number of rows and columns in the dealer dataset
# of rows: 4153
# of columns: 23

----//----//----//----//----//----//----//----
Number of rows and columns in the retail dataset
# of rows: 134261
# of columns: 21

Fixing the date for each dataset
Finished the date fixing functions in 18.19 seconds 

----//----//----//----//----//----//----//----
Adjusting city names in the dealer dataset
Finished the city adjusting functions in  0.01 seconds 

----//----//----//----//----//----//----//----
General cleaning in both datasets
Finished the general cleaning in  0.11 seconds 

----//----//----//----//----//----//----//----
Merging datasets


<ipython-input-4-bd2614c9f61e>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dealer['CITY_DEALER'].fillna('no city found', inplace=True)


Finished the data merging in  0.18 seconds 



In [13]:
new_merged_file = '/content/sample_data/NEW_MERGED.csv'
df_merged.to_csv(new_merged_file, index=False)